In [4]:
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [5]:
#считывание данных 
train_data = pd.read_csv("../../data_anon/hack_offline_train_44fz_223fz_data_anon.csv", sep=';')
train_labels = pd.read_csv("../../data_anon/hack_offline_train_44fz_223fz_labels_anon.csv", sep=';')
test_data = pd.read_csv("../../data_anon/hack_offline_test_44fz_223fz_data_anon.csv", sep=';')

In [6]:
#Заполнение пропусков строкой 'None'
train_data = train_data.fillna('None')
test_data = test_data.fillna('None')

<hr>

# Подготовка данных 

#### Обработка okpd2 и additional_code

In [7]:
train_data['okpd2_or_additional_code'] = train_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)
test_data['okpd2_or_additional_code'] = test_data[['okpd2_code', 'additional_code']].apply(lambda x: x[0] if x[1] == 'None' else x[1], axis=1)

In [8]:
#Просмотр частоты значений 
train_data['okpd2_or_additional_code'].value_counts()

drug    65646
32.5    36292
71.1    17836
20.5    15591
26.2    14783
        ...  
78.2        1
65.2        1
59.0        1
70          1
98          1
Name: okpd2_or_additional_code, Length: 657, dtype: int64

#### Формирование описания тендера

In [9]:
train_data['text_description_tender'] = train_data['purchase_name'] + " " + train_data['lot_name'] + " " + train_data['okpd2_names'] + " " + train_data['additional_code_names'] + " " + train_data['item_descriptions']

In [10]:
train_data.head(3)

,pn_lot_anon,fz,region_code,min_publish_date,purchase_name,lot_name,lot_price,okpd2_code,okpd2_names,additional_code,additional_code_names,item_descriptions,okpd2_or_additional_code,text_description_tender
0,pn_lot_7031618,44fz,2,2019-08-26,Услуги по проведению финансового аудита,None,123500.0,69.2,Услуги по проведению финансового аудита,None,None,Услуги по проведению финансового аудита,69.2,Услуги по проведению финансового аудита None У...
1,pn_lot_7808247,44fz,2,2019-03-12,Приобретение призов для проведения конкурса по...,None,144000.0,26.4,"Телефоны головные, наушники и комбинированные ...",None,None,"Телефоны головные, наушники и комбинированные ...",26.4,Приобретение призов для проведения конкурса по...
2,pn_lot_7009496,44fz,2,2019-03-14,приобретение призов для проведения мероприятия...,None,124200.0,26.2,Устройства запоминающие внешние,None,None,Устройства запоминающие внешние,26.2,приобретение призов для проведения мероприятия...


#### Работа с историей участий поставщика

In [21]:
inn_kpp_history = pd.merge(train_labels, train_data[['pn_lot_anon','region_code', 'okpd2_or_additional_code']], on=['pn_lot_anon'])

In [22]:
inn_kpp_history.head()

,pn_lot_anon,participant_inn_kpp_anon,is_winner,fz,region_code,okpd2_or_additional_code
0,pn_lot_1770702,inn_kpp_3661899,1,44fz,1,42.1
1,pn_lot_1058704,inn_kpp_3661899,1,44fz,1,42.9
2,pn_lot_4186044,inn_kpp_3661899,1,44fz,1,42.1
3,pn_lot_6882961,inn_kpp_3661899,1,44fz,1,42.9
4,pn_lot_5821609,inn_kpp_3661899,1,44fz,1,42.1


In [23]:
#группировка по поставщику 
inn_kpp_history = inn_kpp_history.groupby('participant_inn_kpp_anon').apply(lambda x: [
    list(x['pn_lot_anon']),
    list(x['is_winner']), 
    list(x['fz']), 
    list(x['region_code']), 
    list(x['okpd2_or_additional_code'])]).apply(pd.Series)

In [24]:
inn_kpp_history = inn_kpp_history.reset_index()
inn_kpp_history.columns = ['participant_inn_kpp_anon', 'list_pn_lot_anon',
                           'list_is_winner', 'list_fz', 'list_region_code',
                           'list_okpd2_or_additional_code']

In [25]:
inn_kpp_history.head()

,participant_inn_kpp_anon,list_pn_lot_anon,list_is_winner,list_fz,list_region_code,list_okpd2_or_additional_code
0,inn_kpp_1000018,"[pn_lot_5330449, pn_lot_648665, pn_lot_81155, ...","[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[64, 50, 50, 77, 50, 64, 77, 77, 77, 77, 50, 7...","[58.2, 26.2, 26.4, 26.3, 26.2, 58.2, 26.2, 27...."
1,inn_kpp_1001268,"[pn_lot_7608031, pn_lot_9976372, pn_lot_671651...","[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[77, 79, 79, 79, 79, 27, 79, 79, 79, 79, 79, 7...","[86.9, 86.1, 86.2, 86.9, 86.9, 86.9, 86.9, 86...."
2,inn_kpp_1002518,"[pn_lot_5287076, pn_lot_7474816, pn_lot_121000...","[0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[41, 27, 82, 65, 65, 65, 41, 41, 41, 41, 82, 4...","[30.3, 27.1, 27.1, 25.1, 30.9, 29.2, 25.9, 25...."
3,inn_kpp_1003261,[pn_lot_2647783],[1],[223fz],[54],[28.2]
4,inn_kpp_1003291,"[pn_lot_7794721, pn_lot_4767800, pn_lot_915979...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44fz, 44f...","[50, 12, 50, 50, 51, 69, 11, 12, 11, 12, 12, 1...","[32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 32.5, 21...."


#### Рекомендательная система

Выбирается 35 случайный процедур из подвыборки с совпадением региона и ОКПД2 кода актуальной процедуры с регионами и ОКПД2 кодами из истории участия поставщика

In [16]:
inn_kpp_recommendation = []
similarity_score = 1
for inn_kpp in tqdm_notebook(inn_kpp_history.values):
    participant_inn_kpp_anon, list_participant_inn_kpp_anon, list_is_winner, list_fz, list_region_code, list_okpd2_or_additional_code = inn_kpp
    #подвыборка с совпадением региона и ОКПД2 кода актуальной с историей поставщика
    recommendation = test_data[test_data['region_code'].isin(list_region_code) & test_data['okpd2_or_additional_code'].isin(list_okpd2_or_additional_code)]
    if recommendation.shape[0] >= 35:
        #выбор 35 случайных актуальных процедур из подвыборки
        recommendation = recommendation.sample(35)['pn_lot_anon'].values
        for actual_pn_lot in recommendation:
            inn_kpp_recommendation.append([participant_inn_kpp_anon, actual_pn_lot, similarity_score])

#### Пример формирования файла рекомендаций

In [17]:
recommendation = pd.DataFrame(inn_kpp_recommendation, columns=['inn_kpp', 'actual_recommended_pn_lot', 'similarity_score'])
recommendation.to_csv("team_name.csv", index=False, sep=';')

In [18]:
recommendation

,inn_kpp,actual_recommended_pn_lot,similarity_score
0,inn_kpp_1000018,pn_lot_2745501,1
1,inn_kpp_1000018,pn_lot_5989719,1
2,inn_kpp_1000018,pn_lot_113068,1
3,inn_kpp_1000018,pn_lot_6384714,1
4,inn_kpp_1000018,pn_lot_522774,1
...,...,...,...
452020,inn_kpp_9999743,pn_lot_5500843,1
452021,inn_kpp_9999743,pn_lot_2220520,1
452022,inn_kpp_9999743,pn_lot_576603,1
452023,inn_kpp_9999743,pn_lot_7726061,1


# Подсчёт метрик

In [30]:
test_labels = pd.read_csv("ваша тестирующая выборка", sep=';')

In [31]:
true = set((test_labels['pn_lot_anon'] + "_" + test_labels['participant_inn_kpp_anon']).values)

In [32]:
pred = set((recommendation['actual_recommended_pn_lot'] + "_" + recommendation['inn_kpp']).values)

In [33]:
intersection = len(true.intersection(pred))

In [36]:
print(f"Точность: {intersection / len(pred) * 100}")
print(f"Полнота: {intersection / len(true) * 100}")

Точность: 1.537968032741552
Полнота: 6.389999540420057
